In [1]:
'''
Author: xushengyichn 54436848+xushengyichn@users.noreply.github.com
Date: 2023-01-24 14:28:48
LastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com
LastEditTime: 2023-01-25 18:51:17
FilePath: \20230124优化问题\bayesian.ipynb
Description: 创建贝叶斯优化，优化模态1

Copyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. 
'''


'\nAuthor: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nDate: 2023-01-24 14:28:48\nLastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nLastEditTime: 2023-01-25 18:51:17\nFilePath: \x8230124优化问题\x08ayesian.ipynb\nDescription: 创建贝叶斯优化，优化模态1\n\nCopyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. \n'

In [2]:
#%% 1. 导入库
import time
import matlab
import matlab.engine
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score,KFold,cross_validate
from sklearn.linear_model import LogisticRegression

In [3]:
#%% 2. 定义函数
eng = matlab.engine.start_matlab()
def black_box_function(fTMD1,dTMD1,xTMD1):
        total_tmd_mass_ratio = 0.02 # 总质量比 The total mass ratio
        mass_six_span = 10007779.7 # 深中通道非通航桥六跨连续梁质量 The mass of 6-span continuous beam of the non-navigational bridge of the Zhenzhong-Link
        total_tmd_mass = total_tmd_mass_ratio * mass_six_span # 总质量 The total mass
        
        mTMD1=total_tmd_mass/6 # 质量 The mass mTMD1
        fTMD1=0.7+fTMD1*0.3 # 频率 The frequency fTMD1
        dTMD1=0.05+dTMD1*0.15 # 阻尼比 The damping ratio dTMD1
        xTMD1=xTMD1*660 # TMD1的x坐标 The x-coordinate of TMD1
        t_length=matlab.double(100) # 时间长度 The time length
        number_of_modes_to_control=matlab.double([1]) # 控制模态 The controlled modes
        number_of_modes_to_consider=10 # 考虑模态 The considered modes
        number_of_tmds=1 # TMD数量 The number of TMDs
        modal_damping_ratios=np.ones((1,number_of_modes_to_consider))*0.003 # 模态阻尼比 The modal damping ratios
        
        result = -eng.c_mode1(number_of_modes_to_control,number_of_modes_to_consider,number_of_tmds,modal_damping_ratios,t_length,mTMD1,fTMD1,dTMD1,xTMD1)
        return result

In [4]:
#%% 3. 定义优化器
optimizer = BayesianOptimization(
        f=black_box_function,
        pbounds={"fTMD1": (0,1),
                "dTMD1": (0,1),
                "xTMD1": (0,1),
                },
        verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
        random_state=1,
    )

optimizer.maximize(
        init_points=10,
        n_iter=100,
        acq="ei",  # Expected Improvement.
        # kappa=7.2088954429132,
        xi=0.0012265019946638392,
        # What follows are GP regressor parameters
        kernel=Matern(nu=1.5),
        # alpha=1,
        # alpha=2.2492215663634233e-10,
        alpha=8.960456511761172e-06,
        normalize_y=True,
        n_restarts_optimizer=5,
    )

|   iter    |  target   |   dTMD1   |   fTMD1   |   xTMD1   |
-------------------------------------------------------------
| 1         | -0.2102   | 0.417     | 0.7203    | 0.0001144 |
| 2         | -0.1655   | 0.3023    | 0.1468    | 0.09234   |
| 3         | -0.08284  | 0.1863    | 0.3456    | 0.3968    |
| 4         | -0.1988   | 0.5388    | 0.4192    | 0.6852    |
| 5         | -0.204    | 0.2045    | 0.8781    | 0.02739   |
| 6         | -0.1085   | 0.6705    | 0.4173    | 0.5587    |
| 7         | -0.1944   | 0.1404    | 0.1981    | 0.8007    |
| 8         | -0.1962   | 0.9683    | 0.3134    | 0.6923    |
| 9         | -0.1583   | 0.8764    | 0.8946    | 0.08504   |
| 10        | -0.2009   | 0.03905   | 0.1698    | 0.8781    |
| 11        | -0.1099   | 0.1944    | 0.3465    | 0.376     |
| 12        | -0.08617  | 0.1941    | 0.3264    | 0.3995    |
| 13        | -0.07795  | 0.1746    | 0.3421    | 0.4285    |
| 14        | -0.1935   | 0.2275    | 0.1556    | 0.9473    |
| 15    

In [6]:
optimizer.max['target']
print(optimizer.max)

{'target': -0.0, 'params': {'dTMD1': 0.0, 'fTMD1': 0.5716859074347506, 'xTMD1': 0.0803761444850651}}


In [7]:
for i, res in enumerate(optimizer.res):
    print("Iteration {}: \n\t{}".format(i, res))

Iteration 0: 
	{'target': -0.2101792530827851, 'params': {'dTMD1': 0.417022004702574, 'fTMD1': 0.7203244934421581, 'xTMD1': 0.00011437481734488664}}
Iteration 1: 
	{'target': -0.1654924161080541, 'params': {'dTMD1': 0.30233257263183977, 'fTMD1': 0.14675589081711304, 'xTMD1': 0.0923385947687978}}
Iteration 2: 
	{'target': -0.08283802368547888, 'params': {'dTMD1': 0.1862602113776709, 'fTMD1': 0.34556072704304774, 'xTMD1': 0.39676747423066994}}
Iteration 3: 
	{'target': -0.19881911297042681, 'params': {'dTMD1': 0.538816734003357, 'fTMD1': 0.4191945144032948, 'xTMD1': 0.6852195003967595}}
Iteration 4: 
	{'target': -0.2040331244213868, 'params': {'dTMD1': 0.20445224973151743, 'fTMD1': 0.8781174363909454, 'xTMD1': 0.027387593197926163}}
Iteration 5: 
	{'target': -0.10848895186119137, 'params': {'dTMD1': 0.6704675101784022, 'fTMD1': 0.41730480236712697, 'xTMD1': 0.5586898284457517}}
Iteration 6: 
	{'target': -0.19444608149428064, 'params': {'dTMD1': 0.14038693859523377, 'fTMD1': 0.19810148908